#  Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from collections import Counter
import tensorflow as tf
from tensorflow.keras import Model, models, layers, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load Data

In [2]:
train = pd.read_csv('2_all_data.csv')
test = train.copy()

In [3]:
train.drop(['Timestamp', 'Unnamed: 0'], inplace=True, axis=1)
test.drop(['Timestamp', 'Unnamed: 0'], inplace=True, axis=1)

# Transform to Series Data 

In [4]:
shift_by = -5
n_features = train.shape[1]

In [5]:
# LSTM 모델은 (samples, timesteps, feature)에 해당하는 3d 차원의 shape을 가지므로, 데이터를 시퀀스 형태로 변환

def temporalize(X, timesteps):
    output_X = []
    for i in range(len(X) - timesteps - 1):
        t = []
        for j in range(1, timesteps + 1):
            # Gather the past records upto the lookback period
            t.append(X.loc[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))

In [6]:
timesteps = abs(shift_by)

# Temporalize
x = temporalize(train, timesteps)
print(x.shape)

(210234, 5, 34)


# Split Train / Valid / Test

In [8]:
# Split into train, valid, and test 
x_train = x
x_test = temporalize(test, timesteps)
x_train, x_valid = train_test_split(x_train, test_size=0.2)

print(len(x_train))  
print(len(x_valid)) 
print(len(x_test))

168187
42047
210234


# Standardize

In [9]:
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [10]:
scaler = RobustScaler().fit(flatten(x_train))

x_train_scaled = scale(x_train, scaler)
x_valid_scaled = scale(x_valid, scaler)
x_test_scaled = scale(x_test, scaler)

# Training LSTM Autoencoder

In [11]:
epochs = 20
batch = 128
lr = 0.001

In [12]:
# earlystopping
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=2)

lstm_ae = models.Sequential()
# Encoder
lstm_ae.add(layers.LSTM(128, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_ae.add(layers.LSTM(64, activation='relu', return_sequences=False))
lstm_ae.add(layers.RepeatVector(timesteps))
# Decoder
lstm_ae.add(layers.LSTM(64, activation='relu', return_sequences=True))
lstm_ae.add(layers.LSTM(128, activation='relu', return_sequences=True))
lstm_ae.add(layers.TimeDistributed(layers.Dense(n_features)))

lstm_ae.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-10 09:50:27.028874: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-10 09:50:27.030321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 128)            83456     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 5, 64)            0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 5, 64)             33024     
                                                                 
 lstm_3 (LSTM)               (None, 5, 128)            98816     
                                                                 
 time_distributed (TimeDistr  (None, 5, 34)            4386      
 ibuted)                                                

In [13]:
# compile
lstm_ae.compile(loss="mse", optimizer=optimizers.Adam(lr))

# fit
history = lstm_ae.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=epochs,
    batch_size=batch,
    validation_data=(x_valid_scaled, x_valid_scaled),
    callbacks=[callback],
)

Epoch 1/20


2022-05-10 09:50:31.047914: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-10 09:50:31.879647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1314/1314 [==============================] - ETA: 0s - loss: 0.1112

2022-05-10 09:54:05.157705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1314/1314 [==============================] - 221s 167ms/step - loss: 0.1112 - val_loss: 0.0611
Epoch 2/20
1314/1314 [==============================] - 231s 176ms/step - loss: 0.0689 - val_loss: 0.0663
Epoch 3/20
1314/1314 [==============================] - 245s 186ms/step - loss: 0.0644 - val_loss: 0.0694
Epoch 4/20
1314/1314 [==============================] - 288s 219ms/step - loss: 0.0596 - val_loss: 0.0517
Epoch 5/20
1314/1314 [==============================] - 261s 198ms/step - loss: 0.0583 - val_loss: 0.0502
Epoch 6/20
1314/1314 [==============================] - 281s 214ms/step - loss: 0.0566 - val_loss: 0.0566
Epoch 7/20
1314/1314 [==============================] - 287s 218ms/step - loss: 0.0565 - val_loss: 0.0544
Epoch 8/20
1314/1314 [==============================] - 306s 233ms/step - loss: 0.0580 - val_loss: 0.0534
Epoch 9/20
1314/1314 [==============================] - 286s 218ms/step - loss: 0.0544 - val_loss: 0.0543
Epoch 10/20
1314/1314 [==============================] - 

# threshold

In [14]:
threshold = 0.2

In [15]:
test.shape

(210240, 34)

In [16]:
test_predictions = lstm_ae.predict(x_test_scaled)

2022-05-10 10:57:41.325203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [17]:
mse = np.sqrt(np.mean(np.power(test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2]) - x_test_scaled.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2]), 2)))
print(mse)

0.217048344446869


In [18]:
predict = []
for i in range(len(x_test_scaled)):
    if np.sqrt(np.mean(np.power(test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2])[i] - x_test_scaled.reshape(test_predictions.shape[0], test_predictions.shape[1] * test_predictions.shape[2])[i], 2))) > threshold:
        predict.append(1)
    else:
        predict.append(0)

In [19]:
predict = pd.DataFrame(predict, columns=['Prediction'])
predict.value_counts()

Prediction
0             180006
1              30228
dtype: int64

In [20]:
list0 = list(predict[predict['Prediction'] == 1].index)

In [21]:
test.drop(list0).to_csv('2_only_normal_threshold02.csv', mode='w')

In [22]:
threshold_01 = predict[predict['Prediction']==0]